In [ ]:
import random
import gymnasium as gym # pip install gymnasium[classic-control]

import numpy as np
from keras.layers import Dense
from keras.optimizers import Adam
from keras import Model

import tensorflow as tf

In [ ]:
class DQN(Model):
    def __init__(self):
        super(DQN, self).__init__()
        self.d1 = Dense(64, input_dim=4, activation='tanh')
        self.d2 = Dense(2, activation='linear')
        self.optimizer = Adam(0.001)

        self.M = []  # M은 리플레이 버퍼

    def call(self, x): # x는 넘파이 어레이
        x = self.d1(x)
        y_hat = self.d2(x)
        return y_hat  # y_hat은 텐서 (-1x2)
    
    def remember(self, state, action, reward, next_state, done):
        self.M.append((state, action, reward, next_state, done))

In [ ]:

def update_model():
    global model

    # 리플레이 버퍼가 1000개 이하이면 아직 충분히 쌓이지 않은 것이므로 업데이트하지 않음
    if len(model.M) < 1000:
        return
    
    # 리플레이 버퍼가 10000개 이상이면 가장 오래된 데이터를 지움
    if len(model.M) > 10000:
        model.M.pop(0)

    
    # M에서 랜덤하게 32개의 데이터셋을 뽑음
    batch = random.sample(model.M, 32)

    # 업데이트 수식을 한번에 하기 위해 자료의 구조를 변환
    states = np.array([x[0] for x in batch])  # 32 x 4 어레이, 32는 데이터셋 크기
    actions = np.array([x[1] for x in batch])  # 32 x 2 어레이
    rewards = np.array([x[2] for x in batch])  # 32 x 1 어레이
    next_states = np.array([x[3] for x in batch])  # 32 x 4 어레이
    dones = np.array([x[4] for x in batch])  # 32 x 1 어레이 (done이면 1, 아니면 0)

    # 32개의 데이터셋을 넣었으니 32개의 예측값(Q-value)이 나옴, 32 x 2 어레이
    target_y = model.call(states).numpy()
    # target_y는 Q(s), 32개의 데이터
    # target_y[:, actions]는 Q(s, a), 32개의 데이터
    # verbose는 터미널에 출력되는 로그를 조절하는 옵션, 0은 출력하지 않음

    # action이 수행된 Q-value를 수식에 맞게 업데이트: Q(s, a) = r + γmaxa'Q(s', a')
    # action이 수행되지 않은 나머지 Q-value는 업데이트하지 않음 (predict로 얻은 값 그대로 사용)
    # 이 과정을 32개의 데이터에 한 번에 적용
    target_y[range(32), actions] = rewards + (1 - dones) * 0.95 * np.max(model.call(next_states).numpy(), axis=1)
    # done이면 다음 상태가 없으므로 Q(s', a')는 0

    # 32개의 데이터셋을 한번에 넣어서 한번에 학습
    # 오차는 target_y와 model.predict(states)의 차이
    with tf.GradientTape() as tape:
        loss = tf.reduce_mean(tf.square(target_y - model.call(states)))
    grads = tape.gradient(loss, model.trainable_variables)
    model.optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [ ]:
model = DQN()

# 카트폴 게임 환경 생성
env = gym.make('CartPole-v1')

In [ ]:
for episode in range(200):
    # env.reset의 반환 구조는 state와 info로 이루어져 있음
    # state는 게임 상태를 나타내는 4개의 값으로 이루어진 리스트
    state, info = env.reset()

    # 게임이 끝날 때까지 반복
    for step in range(1000):
        # 1x4 넘파이 어레이로 만들기 위해 리스트([state])로 감싸줌, 1은 데이터셋의 크기
        # predict 함수는 1x2 어레이를 반환, [0]을 붙여서 데이터셋의 한 데이터를 가져옴
        action_list = model.call(np.array([state])).numpy()[0]
        # action_list는 2개의 값으로 이루어진 리스트
        # 각 값은 왼쪽으로 이동할 확률과 오른쪽으로 이동할 확률

        # 출력 층에서 소프트맥스를 사용하지 않았으므로
        # 소프트맥스를 사용하여 선택 확률로 변환
        action_list = np.exp(action_list) / np.sum(np.exp(action_list))
        # action_list의 값으로 확률적으로 액션을 선택
        action = np.random.choice([0, 1], p=action_list)

        # 액션을 취하고 다음 상태, 보상, 게임 종료 여부를 받음
        next_state, reward, done, _, _ = env.step(action)

        # 게임 종료 시 보상을 -10으로 설정
        if done:
            reward = -10
        
        # 매 step마다 리플레이 버퍼에 데이터 추가
        model.M.append((state, action, reward, next_state, done))

        # 매 step마다 모델 업데이트
        update_model()

        # 다음 상태를 현재 상태로 설정
        # 리플레이 버퍼 뒤에 있어야 함
        state = next_state

        # 게임이 끝나면 반복문을 빠져나감
        if done:
            print("Episode: {}, score: {}".format(episode, step))
            break


In [ ]:
model.save_weights('model', save_format='tf')